In [1]:
from math import sqrt

import lightgbm as lgb
import pandas as pd
from catboost import CatBoostRegressor
from deepchem.metrics import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
def metrics(model, X, y):
    y_pred = model.predict(X)
    rmse = sqrt(mean_squared_error(y, y_pred))
    print('RMSE: %f' % rmse)
    mae = mean_absolute_error(y, y_pred)
    print('MAE: %f' % mae)
    r2 = r2_score(y, y_pred)
    print('R2: %f' % r2)
    pearson = pearsonr(y, y_pred)
    print('Pearson: %f' % pearson)

In [3]:
df = pd.read_csv('pdXY_rdkit_descriptors_200ft.csv')
# Extract rows where code is pred into another df
sync = df[df['train_test'] == 'pred']
# Remove rows where code is pred
df = df[df['train_test'] != 'pred']
# Drop train_test column
df = df.drop('train_test', axis=1)
df = df.drop('code', axis=1)
df.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,smiles_len
0,8.645556,0.169259,8.645556,0.169259,0.490728,110.112,104.064,110.036779,42.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0_,C1=CC(=CC=C1O)O,-12.21,15
1,8.975183,0.177713,8.975183,0.177713,0.794127,265.362,248.226,265.111759,96.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCO,-4.62,34
2,10.660372,-4.412252,10.660372,0.026376,0.520728,345.341,327.197,345.078089,120.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)O,-5.39,43
3,11.405303,-5.115401,11.405303,0.245126,0.351107,425.320,406.168,425.044420,144.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,3_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)OP(=...,-9.61,52
4,8.541389,0.225231,8.541389,0.225231,0.669277,143.211,134.139,143.040485,50.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,4_,CC1=C(SC=N1)CCO,-3.48,15


In [4]:
# encode the SMILES strings to categorical values
le = LabelEncoder()
df['smiles'] = le.fit_transform(df['smiles'])

# remove _ at the end of column id and convert to int
df['id'] = df['id'].str.replace('_','').astype(int)

# Split X and y where the dG is the target
X = df.drop('dG', axis=1)
y = df['dG']

# Split train test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 0.0583933 , -0.04460835,  0.0583933 , ..., -0.82493055,
         0.01831198, -1.29653196],
       [ 0.56167447, -0.15965239,  0.56167447, ...,  1.00594357,
         0.0318679 ,  0.23367727],
       [ 1.22061204, -0.25825254,  1.22061204, ...,  0.25986237,
         1.54109301,  2.11974911],
       ...,
       [-0.30878601,  0.78843885, -0.30878601, ..., -0.49995039,
         0.91300232, -0.47804795],
       [ 1.25697854, -0.26479181,  1.25697854, ...,  0.25528518,
         1.52301846,  2.44002546],
       [ 0.44265924, -0.43351015,  0.44265924, ..., -1.26891753,
        -1.64906546, -0.40687543]])

In [6]:
# Check NaN values in X_train
pd.DataFrame(X_train).isnull().sum()
# Replace NaN values with 0
X_train = pd.DataFrame(X_train).fillna(0)
X_test = pd.DataFrame(X_test).fillna(0)

In [9]:
len(X_train)

609

In [10]:
len(X_test)

153

# XGBoost

In [19]:
# Fit an XGBRegressor model
xgb = XGBRegressor(max_depth=100, n_estimators=2000, learning_rate=0.1, min_child_weight=1, subsample=0.8, colsample_bytree=0.8, gamma=0.1, reg_alpha=0.1, reg_lambda=0.1)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.1, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=100, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=2000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [20]:
# Predict the test set and calculate the accuracy
metrics(xgb, X_test, y_test)

RMSE: 1.498628
MAE: 1.042112
R2: 0.528366
Pearson: 0.731765


# Linear Regression

In [21]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [22]:
# Predict the test set and calculate the accuracy
metrics(lr, X_test, y_test)

RMSE: 4.361776
MAE: 2.127011
R2: -2.995246
Pearson: 0.221917


# CatBoost

In [23]:
cat = CatBoostRegressor()
cat.fit(X_train, y_train)

Learning rate set to 0.037857
0:	learn: 2.2925222	total: 146ms	remaining: 2m 25s
1:	learn: 2.2718670	total: 153ms	remaining: 1m 16s
2:	learn: 2.2471879	total: 160ms	remaining: 53.2s
3:	learn: 2.2265299	total: 167ms	remaining: 41.6s
4:	learn: 2.2097520	total: 174ms	remaining: 34.6s
5:	learn: 2.1884163	total: 180ms	remaining: 29.8s
6:	learn: 2.1705005	total: 190ms	remaining: 26.9s
7:	learn: 2.1508855	total: 197ms	remaining: 24.4s
8:	learn: 2.1236736	total: 205ms	remaining: 22.5s
9:	learn: 2.1046479	total: 212ms	remaining: 21s
10:	learn: 2.0846209	total: 220ms	remaining: 19.7s
11:	learn: 2.0661652	total: 227ms	remaining: 18.7s
12:	learn: 2.0500033	total: 237ms	remaining: 18s
13:	learn: 2.0281696	total: 246ms	remaining: 17.3s
14:	learn: 2.0137062	total: 255ms	remaining: 16.8s
15:	learn: 1.9950932	total: 264ms	remaining: 16.2s
16:	learn: 1.9784757	total: 272ms	remaining: 15.7s
17:	learn: 1.9645289	total: 280ms	remaining: 15.3s
18:	learn: 1.9431262	total: 288ms	remaining: 14.9s
19:	learn: 1.

In [24]:
# Predict the test set and calculate the accuracy
metrics(cat, X_test, y_test)

RMSE: 1.451579
MAE: 1.035624
R2: 0.557515
Pearson: 0.751668


# SVR

In [25]:
# Fit an SVR model
svr = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr.fit(X_train, y_train)

SVR(C=1000.0, gamma=0.1)

In [26]:
# Predict the test set and calculate the accuracy
metrics(svr, X_test, y_test)

RMSE: 1.783255
MAE: 1.365067
R2: 0.332204
Pearson: 0.610178


# Random Forest

In [27]:
rf = RandomForestRegressor(n_estimators=1000, max_depth=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=100, n_estimators=1000, random_state=42)

In [28]:
# Predict the test set and calculate the accuracy
metrics(rf, X_test, y_test)

RMSE: 1.480722
MAE: 1.062004
R2: 0.539570
Pearson: 0.745555


# Gradient Boosting

In [29]:
gb = GradientBoostingRegressor(n_estimators=1000, max_depth=100, random_state=42)
gb.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=100, n_estimators=1000, random_state=42)

In [30]:
# Predict the test set and calculate the accuracy
metrics(gb, X_test, y_test)

RMSE: 2.093648
MAE: 1.396960
R2: 0.079499
Pearson: 0.583017


# K-Nearest Neighbors

In [31]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsRegressor()

In [32]:
# Predict the test set and calculate the accuracy
metrics(knn, X_test, y_test)

RMSE: 1.660513
MAE: 1.193190
R2: 0.420969
Pearson: 0.681872


# LightGBM

In [7]:
lgbm = lgb.LGBMRegressor()
lgbm.fit(X_train, y_train)

LGBMRegressor()

In [8]:
metrics(lgbm, X_train, y_train)

RMSE: 0.309805
MAE: 0.194317
R2: 0.982089
Pearson: 0.991939


In [9]:
# Predict the test set and calculate the accuracy
metrics(lgbm, X_test, y_test)

RMSE: 1.441519
MAE: 1.021037
R2: 0.563628
Pearson: 0.755566


# Generate dG for prediction data

In [10]:
sync = sync.drop('train_test', axis=1)
sync = sync.drop('code', axis=1)
sync.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,smiles_len
762,12.541088,-1.196134,12.541088,0.092849,0.574622,336.343,320.215,336.099774,126.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,762_,C12=C(C=C(O1)C(C)(C)O)C1=C(C(=C2)O)C(=O)C=C(O1...,NaN,58
763,12.480255,-1.108454,12.480255,0.108322,0.593099,320.344,304.216,320.104859,120.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,763_,C12=C(C=C(O1)C(C)(C)O)C1=C(C=C2)C(=O)C=C(O1)C1...,NaN,55
764,12.247806,-0.047960,12.247806,0.047960,0.515864,262.264,252.184,262.062994,96.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,764_,C12=C(C=CO1)C1=C(C=C2)C(=O)C=C(O1)C1=CC=CC=C1,NaN,45
765,12.308639,-0.281756,12.308639,0.138137,0.573006,278.263,268.183,278.057909,102.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,765_,C12=C(C=CO1)C1=C(C(=C2)O)C(=O)C=C(O1)C1=CC=CC=C1,NaN,48
766,12.547528,-0.138840,12.547528,0.138840,0.554940,292.290,280.194,292.073559,108.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,766_,C12=CC3=C(C(=C1C=CO2)OC)C(=O)C=C(O3)C1=CC=CC=C1,NaN,47


In [13]:
# encode the SMILES strings to categorical values
le = LabelEncoder()
sync['smiles'] = le.fit_transform(sync['smiles'])

# remove _ at the end of column id and convert to int
sync['id'] = sync['id'].str.replace('_','').astype(int)

# Split X and y where the dG is the target
X_sync = sync.drop('dG', axis=1)

# scale the data
scaler = StandardScaler()
X_sync = scaler.fit_transform(X_sync)

# Predict the sync set with the best model
y_sync = lgbm.predict(X_sync)
y_sync

array([ -8.79849731,  -9.00543753,  -6.10313082,  -7.31963374,
        -8.17014008,  -8.8328802 ,  -8.97152617,  -9.47008423,
        -8.80317294,  -9.32882767,  -8.0949764 ,  -9.88547561,
        -9.64779967,  -9.28271051,  -8.82772067,  -8.84518867,
        -6.97156536,  -7.96840672,  -7.96102642,  -9.6151109 ,
        -9.16792638,  -7.5205578 ,  -8.23240531,  -8.07422577,
        -7.96282477,  -8.8303626 ,  -9.4458271 ,  -9.23942972,
        -9.00391621,  -8.06628518,  -9.21314473,  -8.15914136,
        -7.85313988,  -7.85242057,  -8.45666815,  -9.30829031,
        -9.26135446,  -8.56427542,  -8.38595793,  -8.92475227,
        -9.29898507,  -8.08990961,  -7.51213896,  -7.95040261,
        -7.81504787, -10.09846593,  -7.36672717,  -9.20670023,
        -7.43374127,  -8.58070386,  -8.76739393,  -8.631622  ,
        -8.62263758,  -8.32607017,  -7.83752846,  -8.37560727,
        -7.54329796,  -9.9299899 ,  -8.83720869,  -7.32874228,
        -7.88053332,  -8.631622  ,  -8.81253795,  -9.35

In [14]:
# Add dG column to sync df
sync['dG'] = y_sync

In [15]:
sync.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,smiles_len
762,12.541088,-1.196134,12.541088,0.092849,0.574622,336.343,320.215,336.099774,126.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,762,82,-8.798497,58
763,12.480255,-1.108454,12.480255,0.108322,0.593099,320.344,304.216,320.104859,120.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,763,83,-9.005438,55
764,12.247806,-0.047960,12.247806,0.047960,0.515864,262.264,252.184,262.062994,96.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,764,88,-6.103131,45
765,12.308639,-0.281756,12.308639,0.138137,0.573006,278.263,268.183,278.057909,102.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,765,87,-7.319634,48
766,12.547528,-0.138840,12.547528,0.138840,0.554940,292.290,280.194,292.073559,108.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,766,100,-8.170140,47
